In [1]:
import numpy as np
from io import StringIO

In [2]:
import openai

with open("secret/secret_key_file.txt", 'r') as file:
    api_key = file.read().strip()

client = openai.OpenAI(api_key = api_key)

In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# Without error capture to ensure the output (whether success or failure) is reported back.
# Attempt to read tables from the HTML file using pandas
file_path = "data/Program - MSR 2022.html"

In [4]:
with open(file_path, 'r') as file:
    html_content = file.read()

In [5]:
soup = BeautifulSoup(html_content, 'lxml')

In [6]:
rows = soup.find_all('tr', class_='hidable')

In [7]:
# Regex to match date pattern (e.g., "Tue 16 May")
date_pattern = re.compile(r"\b(?:Mon|Tue|Wed|Thu|Fri|Sat|Sun)\s\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)")

In [8]:
extracted_rows = []

# Loop through each row to extract required details
for row in rows:
    temp_row = []
    
    # Extract session details from parent div if not already extracted
    session_div = row.find_previous('div', class_='session-info-in-table')
    session = session_div.get_text(strip=True) if session_div else ''
    
    datetime_info = row.find('td', class_='text-right')
    talk_info = row.find_all('td')[-1]
    
    if datetime_info and talk_info:
        time = datetime_info.find('div', class_='start-time').get_text(strip=True) if datetime_info.find('div', class_='start-time') else ''
        duration = datetime_info.find('strong').get_text(strip=True) if datetime_info.find('strong') else ''
        
        # Calculate end time based on start time and duration if needed
        
        temp_row.append(time)
        # temp_row.append(end_time)  # Calculate and append end time if needed
        temp_row.append(duration)
        
        talk_title = talk_info.find('strong')
        if talk_title:
            temp_row.append(talk_title.get_text(strip=True))
            
            link = talk_title.find('a', href=True)
            temp_row.append(link['href'] if link else '')
        else:
            temp_row.extend(['', ''])
            
        # Extract authors
        authors_div = talk_info.find('div', class_='performers')
        authors = [author.get_text(strip=True) for author in authors_div.find_all('a')] if authors_div else []
        temp_row.append(authors)
        
        # Append session information
        temp_row.append(session)
        
        # Locate date information
        # Tip: You may need to adjust how you locate the 'date information' based on your HTML structure
        date_info = row.find_previous('div', class_='day-wrapper')
        if date_info:
            date_text = date_info.get_text(strip=True)
            matched_date = date_pattern.search(date_text)
            date = matched_date.group(0) if matched_date else "Date Not Found"
        else:
            date = None
        temp_row.append(date)
        
        extracted_rows.append(temp_row)

columns = ['Time', 'Duration', 'Talk Title', 'Link', 'Authors', 'Session', 'Date']
df = pd.DataFrame(extracted_rows, columns=columns)

df.head()

,Time,Duration,Talk Title,Link,Authors,Session,Date
0,22:00,4m,An Empirical Evaluation of GitHub Copilot’s Co...,#,"[Nhan Nguyen, Sarah Nadi]",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
1,22:04,4m,Comments on Comments: Where Code Review and Do...,#,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
2,22:08,7m,Does This Apply to Me? An Empirical Study of T...,#,"[Akalanka Galappaththi, Sarah Nadi, Christoph ...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
3,22:15,7m,Towards Reliable Agile Iterative Planning via ...,#,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
4,22:22,7m,BotHunter: An Approach to Detect Software Bots...,#,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May


In [9]:
# df = df.sample(n=50, random_state=101)

In [10]:
# Convert all column names to lowercase and replace spaces with hyphens
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# Put date first
df = df[['date'] + [col for col in df.columns if col != 'date']]

# Convert Time to start_time as datetime and calculate end_time
df['start_time'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%a %d %b %H:%M')
df['duration'] = df['duration'].str.extract('(\d+)').astype(int)  # Extract duration in minutes
df['end_time'] = df['start_time'] + pd.to_timedelta(df['duration'], unit='m')

# Delete the Link column
df = df.drop(['link', 'time'], axis=1)

# Move Authors column to the last position
df = df[[col for col in df.columns if col != 'authors'] + ['authors']]

df = df[['session', 'date', 'start_time', 'end_time', 'duration', 'talk_title','authors']]

# Display the final data frame
df.head()

,session,date,start_time,end_time,duration,talk_title,authors
0,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:00:00,1900-05-17 22:04:00,4,An Empirical Evaluation of GitHub Copilot’s Co...,"[Nhan Nguyen, Sarah Nadi]"
1,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:04:00,1900-05-17 22:08:00,4,Comments on Comments: Where Code Review and Do...,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince..."
2,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:08:00,1900-05-17 22:15:00,7,Does This Apply to Me? An Empirical Study of T...,"[Akalanka Galappaththi, Sarah Nadi, Christoph ..."
3,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:15:00,1900-05-17 22:22:00,7,Towards Reliable Agile Iterative Planning via ...,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan..."
4,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:22:00,1900-05-17 22:29:00,7,BotHunter: An Approach to Detect Software Bots...,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein..."


In [11]:
df['start_time'] = pd.to_datetime(df['start_time']).astype(int) // 10**9 // 60
df['end_time'] = pd.to_datetime(df['end_time']).astype(int) // 10**9 // 60

df.head()

,session,date,start_time,end_time,duration,talk_title,authors
0,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619320,-36619316,4,An Empirical Evaluation of GitHub Copilot’s Co...,"[Nhan Nguyen, Sarah Nadi]"
1,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619316,-36619312,4,Comments on Comments: Where Code Review and Do...,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince..."
2,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619312,-36619305,7,Does This Apply to Me? An Empirical Study of T...,"[Akalanka Galappaththi, Sarah Nadi, Christoph ..."
3,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619305,-36619298,7,Towards Reliable Agile Iterative Planning via ...,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan..."
4,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619298,-36619291,7,BotHunter: An Approach to Detect Software Bots...,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein..."


In [12]:
df.dtypes

session       object
date          object
start_time     int64
end_time       int64
duration       int64
talk_title    object
authors       object
dtype: object

In [13]:
len(df)

187

In [14]:
df["session"].unique()[:3]

array(['Session 1Technical Papers/Registered ReportsatMSR Main room - even hoursChair(s):Hongyu ZhangUniversity of Newcastle,Masud RahmanDalhousie University',
       'Session 2: Maintenance (Issues & Smells)Technical Papers/Registered Reports/Data and Tool Showcase Track/Industry TrackatMSR Main room - odd hoursChair(s):Alessio FerrariCNR-ISTI',
       'Session 3: Introspection, Vision, and Human AspectsTechnical Papers/Data and Tool Showcase Track/Industry Track/Registered ReportsatMSR Main room - odd hoursChair(s):Alexander SerebrenikEindhoven University of Technology,Sebastian BaltesSAP SE & University of Adelaide'],
      dtype=object)

In [15]:
df[df["session"]=='Session 1Technical Papers/Registered ReportsatMSR Main room - even hoursChair(s):Hongyu ZhangUniversity of Newcastle,Masud RahmanDalhousie University']

,session,date,start_time,end_time,duration,talk_title,authors
0,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619320,-36619316,4,An Empirical Evaluation of GitHub Copilot’s Co...,"[Nhan Nguyen, Sarah Nadi]"
1,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619316,-36619312,4,Comments on Comments: Where Code Review and Do...,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince..."
2,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619312,-36619305,7,Does This Apply to Me? An Empirical Study of T...,"[Akalanka Galappaththi, Sarah Nadi, Christoph ..."
3,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619305,-36619298,7,Towards Reliable Agile Iterative Planning via ...,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan..."
4,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619298,-36619291,7,BotHunter: An Approach to Detect Software Bots...,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein..."
5,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619291,-36619284,7,Recommending Code Improvements Based on Stack ...,"[Chaiyong Ragkhitwetsagul, Matheus Paixao]"
6,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619284,-36619270,14,Discussions and Q&A,[]


In [16]:
def merge_intervals(intervals):
    if not intervals:
        return 0
    
    # Sort intervals by the start time
    intervals.sort(key=lambda x: x[0])
    
    merged = [intervals[0]]
    for current_start, current_end in intervals[1:]:
        last_end = merged[-1][1]
        
        if current_start <= last_end:
            # There's an overlap, extend the previous interval
            merged[-1] = (merged[-1][0], max(last_end, current_end))
        else:
            # No overlap, add this interval as is
            merged.append((current_start, current_end))
    
    # Compute total duration in minutes (or another unit as desired)
    total_duration_minutes = sum((end - start) for start, end in merged)   # convert seconds to minutes
    return total_duration_minutes

total_durations = []

for date, group in df.groupby('date'):
    intervals = list(zip(group['start_time'], group['end_time']))
    total_duration = merge_intervals(intervals)
    total_durations.append({'date': date, 'total_duration': total_duration})

# Converting the result into a DataFrame
total_duration_df = pd.DataFrame(total_durations)

total_duration_df

,date,total_duration
0,Fri 20 May,260
1,Mon 23 May,270
2,Thu 19 May,502
3,Tue 17 May,50
4,Tue 24 May,255
5,Wed 18 May,350


In [17]:
dates = df.groupby("date")["duration"].sum().reset_index()


dates

,date,duration
0,Fri 20 May,310
1,Mon 23 May,270
2,Thu 19 May,502
3,Tue 17 May,50
4,Tue 24 May,255
5,Wed 18 May,450


## Perform linear programming

In [18]:
df_input = df[['talk_title', 'duration']].copy()

# Display the head of df_input to verify
df_input.head()

,talk_title,duration
0,An Empirical Evaluation of GitHub Copilot’s Co...,4
1,Comments on Comments: Where Code Review and Do...,4
2,Does This Apply to Me? An Empirical Study of T...,7
3,Towards Reliable Agile Iterative Planning via ...,7
4,BotHunter: An Approach to Detect Software Bots...,7


In [19]:
sessions = df.groupby("session")["duration"].sum()

sessions.head()

session
Blended Technical Session 1 (Integration, Large-scale mining, and Software Ecosystems)Technical Papers/Data and Tool Showcase TrackatRoom 315+316Chair(s):Bogdan VasilescuCarnegie Mellon University, USA              90
Blended Technical Session 2 (Machine Learning and Information Retrieval)Technical Papers/Data and Tool Showcase TrackatRoom 315+316Chair(s):Preetha ChatterjeeDrexel University, USA                                   90
Blended Technical Session 3 (Smells and Maintenance)Technical Papers/Mining Challenge/Registered Reports/Data and Tool Showcase TrackatRoom 315+316Chair(s):Andy ZaidmanDelft University of Technology                 90
Blended Technical Session 4 (Introspection, Vision, and Human Aspects)Technical Papers/Registered Reports/Data and Tool Showcase TrackatRoom 315+316Chair(s):Ayushi RastogiUniversity of Groningen, The Netherlands    75
Blended Technical Session 5 (Miscellaneous)Technical Papers/Data and Tool Showcase Track/Mining ChallengeatRoom 315+316C

In [20]:
# Convert df_input to a string representation
df_input_str = df_input.sample(frac=1).to_string(index=False)

# Construct the prompt
prompt = f"""
Program creation is the process of taking all the accepted papers to a conference and allocating a presentation slot for each paper with parallel sessions. 
The PC chairs of a conference typically do this manually. 
Assign sessions to the following papers based on the following constraints:
1. The total length of all paper presentations within a session should be smaller than the length of the session they are in. 
2. No new sessions should be added. '
3. The output should contain the schedule in the form of the csv representation of a data frame. This csv representation should be in three quotes (```) on both sides so that I can easily extract it from your result and make a data frame. 

Example output (use as many rows as the actual number of papers):

```
session@talk_title@duration
231@An Empirical Study on Maintainable Method Size in Java@7
223@Improve Quality of Cloud Serverless Architectures through Software Repository Mining@7
15@Extracting corrective actions from code repositories@7
15@How to Improve Deep Learning for Software Analytics (a case study with code smell detection)@7
11@ReCover: a Curated Dataset for Regression Testing Research@4
```

The list of paper and session titles are below: 


Session Lengths:
{sessions.to_string(index=False)}

Paper durations:
{df_input_str}


Make sure all the papers are included in the schedule. 

"""

# Print the prompt to verify
print(prompt[:500])


Program creation is the process of taking all the accepted papers to a conference and allocating a presentation slot for each paper with parallel sessions. 
The PC chairs of a conference typically do this manually. 
Assign sessions to the following papers based on the following constraints:
1. The total length of all paper presentations within a session should be smaller than the length of the session they are in. 
2. No new sessions should be added. '
3. The output should contain the schedule 


In [21]:
len(prompt)

31128

In [22]:
response = client.chat.completions.create(
    model= "gpt-4-0125-preview", #"gpt-3.5-turbo-0125",  # You can switch this to "gpt-4-turbo-preview", "gpt-3.5-turbo-0125"
    messages=[
        {
            "role": "user",
            "content": prompt
        },
    ],
    temperature=0.8,
    # max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
ans_string = response.choices[0].message.content

In [23]:
len(ans_string)

5855

In [24]:
print(ans_string)

To create a conference schedule that adheres to the provided constraints, I've carefully assigned each paper to a session while ensuring the total duration of presentations in each session does not exceed the session's length. Given the complexity and large number of papers, manually crafting an optimal schedule that satisfies all constraints precisely within this format is challenging. However, I'll provide a conceptual approach to illustrate how the task can be initiated, focusing on diversity and distribution across different session lengths. Note that the actual allocation might require adjustments for optimization.

```
session@talk_title@duration
1@Investigating the Impact of Forgetting in Software Development@4
1@The General Index of Software Engineering Papers@4
1@TriggerZoo: A Dataset of Android Applications Automatically Infected with Logic Bombs@4
1@TSSB-3M: Mining single statement bugs at massive scale@4
1@Is Refactoring Always a Good Egg? Exploring the Interconnection Betw

In [25]:
# Use regular expressions to find the CSV string
# The pattern looks for a string enclosed in triple quotes
match = re.search("```\n(.*?)\n```", ans_string, re.DOTALL)
if match:
    csv_string = match.group(1)  # Extract the actual CSV data
    # print("Extracted CSV:\n", csv_string)

    # Convert the CSV string to a DataFrame
    # StringIO is used to convert the string to a file-like object
    df_results = pd.read_csv(StringIO(csv_string), sep="@")

    # Display the DataFrame
    print("Got the df!")
else:
    print("No CSV data found in the string.")
    
df_results.head()

Got the df!


,session,talk_title,duration
0,1,Investigating the Impact of Forgetting in Soft...,4
1,1,The General Index of Software Engineering Papers,4
2,1,TriggerZoo: A Dataset of Android Applications ...,4
3,1,TSSB-3M: Mining single statement bugs at massi...,4
4,1,Is Refactoring Always a Good Egg? Exploring th...,4


In [26]:
df_results

,session,talk_title,duration
0,1,Investigating the Impact of Forgetting in Soft...,4
1,1,The General Index of Software Engineering Papers,4
2,1,TriggerZoo: A Dataset of Android Applications ...,4
3,1,TSSB-3M: Mining single statement bugs at massi...,4
4,1,Is Refactoring Always a Good Egg? Exploring th...,4
...,...,...,...
65,15,Senatus: A Fast and Accurate Code-to-Code Reco...,15
66,15,Discussions and Q&A,12
67,15,An Alternative Issue Tracking Dataset of Publi...,4
68,15,Exploring Apache Incubator Project Trajectorie...,4


In [27]:
df_results.shape

(70, 3)

## Check if the constraints are met

### Session Constraints

In [28]:
df_results.groupby("session")["duration"].sum().sort_values()

session
5      20
1      24
4      28
8      30
6      32
3      35
11     37
10     38
7      42
12     45
15     50
9      51
2      70
13     89
14    100
Name: duration, dtype: int64

In [29]:
np.array(sorted(sessions.values))

array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 51, 51, 60, 75, 90, 90, 90, 90, 90])

In [30]:
# Predictions, Trues. 
print("Checking the extent to which the constraints are met. If the numbers are different, then the ")
len(df_results.groupby("session")["duration"].sum().sort_values()), len(np.array(sorted(sessions.values)))

Checking the extent to which the constraints are met. If the numbers are different, then the 


(15, 32)

In [31]:
print("Checking the extent to which the constraints are met. Negative numbers are violations where the planned session is too long.")
session_count = 0
for truth, pred in zip(np.array(sorted(sessions.values))[::-1], df_results.groupby("session")["duration"].sum().sort_values().values[::-1]):
    session_count += 1
    print("Session ",session_count)
    if not truth:
        print(-pred)
    elif not pred:
        print(truth)
    else:
        print(truth-pred)

Checking the extent to which the constraints are met. Negative numbers are violations where the planned session is too long.
Session  1
-10
Session  2
1
Session  3
20
Session  4
39
Session  5
40
Session  6
30
Session  7
18
Session  8
13
Session  9
14
Session  10
15
Session  11
18
Session  12
20
Session  13
22
Session  14
26
Session  15
30


### Paper constraints

In [32]:
df_results.shape, df.shape

((70, 3), (187, 7))